# Python Data Wrangling with `pandas`

* * * 
<div class="alert alert-success">  
    
### Learning Objectives 
    
* Gain familiarity with `pandas` and the core `DataFrame` object
* Apply core data wrangling techniques in `pandas`
* Understand the flexibility of the `pandas` library
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Pandas can be used for!<br>

Welcome back!

### Sections
4. [Manipulating DataFrames](#variables)
5. [Merging](#variables)
6. [Grouping](#grouping)
7. [Data products](#demo)

Let's start back up by importing our libraries and loading up our data

In [636]:
# Imports pandas and assign it to the variable `pd`
import pandas as pd

# Load matplotlib for plotting later in the notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [637]:
# Open the unemplyment dataset
unemployment = pd.read_csv('../data/country_total.csv')

# Open the countries dataset
countries_url = 'https://raw.githubusercontent.com/dlab-berkeley/Python-Data-Wrangling/main/data/countries.csv'
countries = pd.read_csv(countries_url)

## Manipulating DataFrames
We often times want to make changes to how our data is named, ordered, etc.
### Renaming Columns

In the unemployment dataset, you may have noticed that the `month` column also includes the year:

In [638]:
unemployment.head()

,country,seasonality,month,unemployment,unemployment_rate
0,at,nsa,1993.01,171000,4.5
1,at,nsa,1993.02,175000,4.6
2,at,nsa,1993.03,166000,4.4
3,at,nsa,1993.04,157000,4.1
4,at,nsa,1993.05,147000,3.9


Let's go ahead and rename it to something more descriptive - `year_month`. The [`.rename()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) allows you to modify index labels and/or column names.  

In [639]:
unemployment.rename(columns={'month' : 'year_month'}).head()

,country,seasonality,year_month,unemployment,unemployment_rate
0,at,nsa,1993.01,171000,4.5
1,at,nsa,1993.02,175000,4.6
2,at,nsa,1993.03,166000,4.4
3,at,nsa,1993.04,157000,4.1
4,at,nsa,1993.05,147000,3.9


💡 **Tip**: You'll notice in the code above that we combined multiple methods together, one after another. This is referred to as [method chaining](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) (see bottom left hand corner). The methods execute sequentially, left-to-right.


As you can see, we passed a `dict` to the `columns` parameter, with the original name as the key and the new name as the value.  Note that `.rename()`, like many other `pandas` methods, returns a copy of the full dataframe. This copy is not saved anywhere unless you explicitly assign it to a variable.

In [640]:
# Note that the column name is unchanged
unemployment.columns

Index(['country', 'seasonality', 'month', 'unemployment', 'unemployment_rate'], dtype='object')

In [641]:
unemployment = unemployment.rename(columns={'month' : 'year_month'})

In [642]:
# Note that the column name has now changed
unemployment.columns

Index(['country', 'seasonality', 'year_month', 'unemployment',
       'unemployment_rate'],
      dtype='object')

### Sorting Values

At this point, you might be curious to know what the highest unemployment rates are in this data. For this, we'll use the `sort_values()` method to sort the data.

In [643]:
unemployment.sort_values('unemployment_rate', ascending=False).head()

,country,seasonality,year_month,unemployment,unemployment_rate
15526,pl,nsa,2004.02,3531000,20.9
15525,pl,nsa,2004.01,3520000,20.7
15514,pl,nsa,2003.02,3460000,20.7
5663,es,sa,2010.09,4773000,20.6
15527,pl,nsa,2004.03,3475000,20.6


The above code creates a copy of the `DataFrame`, sorted in *descending* order, and prints the first five rows.

You may have noticed that the data set includes a `seasonality` column, which we haven't yet discussed. The unemployment rate in this data is actually calculated in three separate ways. Let's look at the values.

In [644]:
unemployment['seasonality'].unique()

array(['nsa', 'sa', 'trend'], dtype=object)

The three options above correspond to:

* nsa: not seasonally adjusted
* sa: seasonally adjusted
* trend: trend cycle

For the rest of the lesson, let's focus only on the seasonally adjusted unemployment rate information so that the values are more comparable.

In [645]:
unemployment = unemployment[unemployment['seasonality'] == 'sa']

In [646]:
unemployment['seasonality'].unique()

array(['sa'], dtype=object)

## 🥊 Challenge 5: Boolean Indexing and Sorting
Let's put boolean indexing and sorting together to answer a practical question: What is the all-time highest seasonally adjusted unemployment rate in our data? What country has it? 

This is a complex question, so let's break it into parts:
1. Use boolean indexing to filter the data down to only seasonally adjusted unemployment figures
2. Sort this data based on the unemployment rate
3. Select the top row  

([See here](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Country_codes) for translating country codes to names)

In [647]:
# Answer
unemployment.sort_values('unemployment_rate', ascending=False).head(1)

,country,seasonality,year_month,unemployment,unemployment_rate
5664,es,sa,2010.1,4758000,20.6


### Adding new columns

So, we still want to split `year_month` into two separate columns. Above, we saw that this column is type (technically, `dtype`) `float64`. We'll first extract the year using the `.astype()` method. This allows for type casting&mdash;basically converting from one type to another. We'll then subtract this value from `year_month`&mdash;to get the decimal portion of the value&mdash;and multiply the result by 100 and convert to `int`.

For more information on `pandas` `dtype`s, check the documentation [here](http://pandas.pydata.org/pandas-docs/stable/basics.html#dtypes).

In [648]:
unemployment['year_month'] = unemployment['year_month'].astype(str)
unemployment['year_month'].head(10)

216    1993.01
217    1993.02
218    1993.03
219    1993.04
220    1993.05
221    1993.06
222    1993.07
223    1993.08
224    1993.09
225     1993.1
Name: year_month, dtype: object

In [649]:
(unemployment['year_month'].str.len() == 7).head(10)

216     True
217     True
218     True
219     True
220     True
221     True
222     True
223     True
224     True
225    False
Name: year_month, dtype: bool

In [650]:
unemployment = unemployment[unemployment['year_month'].str.len() == 7]

`pandas` provides a `to_datetime()` function that makes this relatively simple. It converts an argument&mdash;a single value or an array of values&mdash;to `datetime` (note that the return value [depends on the input](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)). If we were interested in March 23, 1868, for example, we could do the following:

In [651]:
pd.to_datetime('1868/3/23')

Timestamp('1868-03-23 00:00:00')

The argument doesn't necessarily have to be specified in the `yyyy/mm/dd` format. You could list it as `mm/dd/yyyy`, but it's a good idea to be explicit. To do so, we pass in a string into the `format` keyword:

In [652]:
pd.to_datetime('3/23/1868', format='%m/%d/%Y')

Timestamp('1868-03-23 00:00:00')

In [653]:
pd.to_datetime('1993.01', format='%Y.%m')

Timestamp('1993-01-01 00:00:00')

In [654]:
pd.to_datetime(unemployment['year_month'].astype(str), format='%Y.%m').head(13)

216   1993-01-01
217   1993-02-01
218   1993-03-01
219   1993-04-01
220   1993-05-01
221   1993-06-01
222   1993-07-01
223   1993-08-01
224   1993-09-01
226   1993-11-01
227   1993-12-01
228   1994-01-01
229   1994-02-01
Name: year_month, dtype: datetime64[ns]

In [655]:
unemployment['date'] = pd.to_datetime(unemployment['year_month'].astype(str), format='%Y.%m')

In [656]:
unemployment['year'] = unemployment['date'].dt.year
unemployment['month'] = unemployment['date'].dt.month

In [657]:
unemployment.head(5)

,country,seasonality,year_month,unemployment,unemployment_rate,date,year,month
216,at,sa,1993.01,144000,3.8,1993-01-01,1993,1
217,at,sa,1993.02,145000,3.8,1993-02-01,1993,2
218,at,sa,1993.03,147000,3.9,1993-03-01,1993,3
219,at,sa,1993.04,149000,3.9,1993-04-01,1993,4
220,at,sa,1993.05,151000,4.0,1993-05-01,1993,5


## Merging DataFrames

So far, our `DataFrame` is organized in a reasonable way. But, we know we can do better. We're eventually going to be interested in the unemployment rate for each country. The trouble is, we don't exactly know what the values in `country` refer to. We can fix that by getting country names. These can be found in `countries.csv`.

In [658]:
countries.head()

,country,google_country_code,country_group,name_en,name_fr,name_de,latitude,longitude
0,at,AT,eu,Austria,Autriche,Österreich,47.696554,13.345980
1,be,BE,eu,Belgium,Belgique,Belgien,50.501045,4.476674
2,bg,BG,eu,Bulgaria,Bulgarie,Bulgarien,42.725674,25.482322
3,hr,HR,non-eu,Croatia,Croatie,Kroatien,44.746643,15.340844
4,cy,CY,eu,Cyprus,Chypre,Zypern,35.129141,33.428682


Because the data we need is stored in two separate files, we'll want to merge the data somehow. Let's determine which column we can use to join this data by taking a look at `unemplyment`

In [659]:
unemployment.head()

,country,seasonality,year_month,unemployment,unemployment_rate,date,year,month
216,at,sa,1993.01,144000,3.8,1993-01-01,1993,1
217,at,sa,1993.02,145000,3.8,1993-02-01,1993,2
218,at,sa,1993.03,147000,3.9,1993-03-01,1993,3
219,at,sa,1993.04,149000,3.9,1993-04-01,1993,4
220,at,sa,1993.05,151000,4.0,1993-05-01,1993,5


These two DataFrames seem to have a similar `country` column, with two character codes for each country. Let's try doing a merge of these two datasets based on the `country` column.

`pandas` includes an easy-to-use merge function. Let's use it to merge the two `DataFrame`s on country code.

In [660]:
unemployment_merged = pd.merge(unemployment, countries, on='country')

In [661]:
unemployment_merged.head()

,country,seasonality,year_month,unemployment,unemployment_rate,date,year,month,google_country_code,country_group,name_en,name_fr,name_de,latitude,longitude
0,at,sa,1993.01,144000,3.8,1993-01-01,1993,1,AT,eu,Austria,Autriche,Österreich,47.696554,13.34598
1,at,sa,1993.02,145000,3.8,1993-02-01,1993,2,AT,eu,Austria,Autriche,Österreich,47.696554,13.34598
2,at,sa,1993.03,147000,3.9,1993-03-01,1993,3,AT,eu,Austria,Autriche,Österreich,47.696554,13.34598
3,at,sa,1993.04,149000,3.9,1993-04-01,1993,4,AT,eu,Austria,Autriche,Österreich,47.696554,13.34598
4,at,sa,1993.05,151000,4.0,1993-05-01,1993,5,AT,eu,Austria,Autriche,Österreich,47.696554,13.34598


In [662]:
unemployment_merged.to_csv("

SyntaxError: EOL while scanning string literal (3283550765.py, line 1)

---

## 🥊 Challenge 6

You may sometimes need to merge on columns with different names. To do so, use the `left_on` and `right_on` parameters, where the first listed `DataFrame` is the "left" one and the second is the "right." It might look something this:

```
pd.merge(one, two, left_on='city', right_on='city_name')
```

Suppose wanted to merge `unemployment` with a new DataFrame called `country_codes`, where the abbreviation for each country is in the column "c_code":

---

In [ ]:
# Making a messier DataFrame
country_codes = countries.rename({"country": "c_code"}, axis=1).drop("country_group", axis=1)
country_codes.head()

Use `merge` to merge `unemployment` and `country_codes` on their country codes. Make sure to specify `left_on=` and `right_on=` in the call to `merge`!

In [ ]:
# YOUR CODE HERE

## Calculating Unique and Missing Values
Next, we should pause for a moment and think about what data we really care about. For our purposes, the variable of interest is `unemployment_rate`. The number of observations by country only reflect the number of instances of each country name in the dataset. It is possible, maybe even expected, to have some missing data within those instances. Let's find out **how many unemployment rate values are missing.**

In [ ]:
unemployment_merged.isnull()

The `.isnull()` method returns a corresponding boolean value for each entry in the unemployment rate `Series`. In Python `True` is equivalent to 1 and `False` is equivalent to 0. Thus, when we add the result (with `.sum()`), we get a count for the *total* number of missing values.

In [ ]:
unemployment_merged.isnull().sum()

In [ ]:
unemployment_merged['unemployment_rate'].isnull()

In [ ]:
missing_data = unemployment_merged[unemployment_merged['unemployment_rate'].isnull()]
missing_data.head()

In [ ]:
unemployment_merged = unemployment_merged.dropna(subset=['unemployment_rate'])

# Grouping and Aggregating Data

What if we'd like to know how many missing values exist at the *country* level? To do so, we need to calculate a separate value for each unique value of country. In other words, we're going to **group** our data **by** a specific column, and calculate some quantity within each group. The "group-by" operation is a fundamental technique used with tabular data.

For simple grouping operations, we can use the handy `.value_counts()` method. We typically run this on a single column, and it will return a table showing how many observations there are for each unique value in the column

In [ ]:
missing_data['name_en'].value_counts()

This tells us that in the `missing_data` DataFrame, Croatia appears 216 times, Bulgaria 180, etc. Each of these figures is the number of missing values for each Country.

## 🥊 Challenge 7
Try using `.value_counts()` on the `missing_data` DataFrame to find out how many missing rows are from EU versus non-EU records

In [ ]:
### Answer
missing_data['country_group'].value_counts()

Cool, now we have a good idea of how the "missing-ness" in the unemployment rate column is distributed: we are missing a lot of data from Croatia, Bulgaria, and Malta, and most of our missing-ness generally is from countries in the EU rather than non-EU.  

Let's turn back to our larger merged DataFrame with all of the data. We might wonder how many of our records generally are from EU versus non-EU countries. We can now easily take a look at that with `.value_counts()`:

In [ ]:
unemployment_merged

But what if we want to do something more complex. For example, **what was the average unemployment rate for EU versus non-EU countries?**. Answering this goes beyond `.value_counts()`, but builds on the same ideas.  

This sort of question is a classic example of **grouping** -- we want to group our data based on whether or not each row is a record from a country in the EU, select a column of interest (in this case `unemployment_rate`), then compute a summary statistic (in this case the average) for each group on that column. Sounds complicated, so let's break it down into pieces.  

We have to use a `pandas` method called `.groupby()` to do this. We want to group based on EU membership, and we can do that as follows:

In [ ]:
unemployment_merged.groupby('country_group')

But `.groupby()` doesn't actually return data or output -- it just groups the data. We now have to select a column of data, then chain on another method that tells `pandas` how we want to **aggregate** the data from that column for each group. For example, below we will group the data, select a column, then average that column's data for each group.

In [ ]:
unemployment_merged.groupby('country_group')['unemployment_rate'].mean()

Dissecting the code, we basically told `pandas`:
1. <code>unemployment_merged<code><mark style="background: yellow">.groupby('country_group')</mark>['unemployment_rate'].mean()</code>

    Group all of our rows based on the unique values of the `country_group` column -- EU, non-EU
    
2. <code>unemployment_merged<code>.groupby('country_group')<mark style="background: yellow">['unemployment_rate']</mark>.mean()</code>

    Select the `unemployment_rate` column

2. <code>unemployment_merged<code>.groupby('country_group')['unemployment_rate']<mark style="background: yellow">.mean()</mark></code>

   Compute the average of the selected column (`unemployment_rate`) for each group

We can confirm this behavior using boolean indexing as well. If we index to only those records from EU countries, select the `unemployment_rate` column, then compute the average, we should get 8.3, the same value computed with groupby:

In [ ]:
unemployment_merged[unemployment_merged['country_group'] == 'eu']['unemployment_rate'].mean()

In [ ]:
unemployment_merged

---

## 🥊 Challenge 8

Find the average unemployment rate for European Union vs. non-European Union countries. 

1. First, use `groupby()` to group on "name_en".
2. Then, select the "unemployment_rate" column,
3. Aggregate by using `.mean()` to get the average.
4. Chain on the method `.sort_values()`. What do you observe?

---

In [ ]:
# YOUR CODE HERE
unemployment_merged.groupby('name_en')['unemployment_rate'].mean().sort_values()

Grouping lends itself well to data visualization. In `pandas`, visualization is as simple as calling the `.plot()` method, then supplying optional arguments (here I supplied `kind="bar"` to make a bar chart rather than the default line-chart). The following is the maximum unemployment rate across countries in the data for each year:

In [ ]:
unemployment_merged.groupby('year')['unemployment_rate'].max().plot(kind="bar")

This isn't such an ideal visualization of these trends. We have access to monthly data, so let's plot that instead.

In [ ]:
unemployment_merged = unemployment_merged.set_index('date')

In [ ]:
unemployment_merged.loc[unemployment_merged['name_en'] == 'Spain', 'unemployment_rate'].plot()

In [ ]:
unemployment_merged.loc[unemployment_merged['name_en'] == 'Spain', 'unemployment_rate'].plot()
unemployment_merged.loc[unemployment_merged['name_en'] == 'Portugal', 'unemployment_rate'].plot()
plt.legend(["Spain", "Portugal"])
plt.show()

***
# Cut Below

## Exporting A DataFrame to a CSV File

From our analysis on missing unemployment rates, we can observe that Croatia is missing a relatively large amount of data (~66% of records are missing). The `unemployment_rate` data frame contains this information, and is useful to keep for further analysis.

Suppose we wanted to save this as a .csv file. For this, we'd use the `.to_csv()` method:

In [ ]:
unemployment_rate.to_csv('../data/unemployment_missing.csv')

Let's import this newly created file, and see what it looks like:

In [ ]:
unemployment_missing = pd.read_csv('../data/unemployment_missing.csv')
unemployment_missing.head()

What do you see here? By default, this method writes the indices. We probably don't want that. Let's edit the code. Let's also be explicit about the type of delimiter we're interested in. (Values can be separated by pipes (`|`), semicolons (`;`), tabs (`\t`), etc.)

In [ ]:
unemployment_rate.to_csv('../data/unemployment_missing.csv', index=False, sep=',')

In [ ]:
unemployment_missing = pd.read_csv('../data/unemployment_missing.csv')
unemployment_missing.head()

Much better!

Let's return to our main `DataFrame`. Now that we have the missing values information in `unemployment_rate`, we can **drop the last column** we added to `unemployment`.

In [ ]:
unemployment.drop('unemployment_rate_null', axis=1, inplace=True)

It's important to specify the `axis` parameter. Specifically, `axis=1` refers to columns while `axis=0` refers to rows. The parameter `inplace=True` simply modifies the actual `DataFrame` rather than returning a new `DataFrame`.

In [ ]:
unemployment.head()

## Dealing With Missing Values: Boolean Indexing

Now that we know about the missing values, we have to deal with them. There are two main options:

* Fill the missing values with some other values.
* Do not use observations with missing values.
    * Depending on the analysis, we may want to exclude entire countries.
    
Because countries with missing unemployment rate data have at least 36 missing values, which is too many to fill, we'll take the second approach and **exclude missing values** from our primary analyses.

Instead of just getting rid of that data, it might make sense to store it in a separate `DataFrame`. This way, we could answer questions such as, "do missing values occur during certain months (or years) more frequently?" With this, we will introduce the concept of *boolean indexing* for filtering data.

Recall that we used the `isnull()` function to determine whether a data element is missing. This function returns a `pd.Series` containing Boolean values:

In [ ]:
unemployment['unemployment_rate'].isnull()

To subselect the rows with missing values, we'll use this Boolean `pd.Series` to index the `unemployment` data frame, just like we might row numbers or column names. Specifically, we'll pass the Boolean series into a pair of bracket, which will then return only the rows where the value in the array is `True`. Let's see what this looks like:

In [ ]:
unemployment_rate_missing = unemployment[unemployment['unemployment_rate'].isnull()]

In [ ]:
unemployment_rate_missing.head()

It is also possible to specify multiple conditions using the `&` operator, but each condition needs to be inside of parentheses. The `.isin()` method, which takes a `list` of values, is useful when you're interested in conditioning on multiple values on a given column. For example, if you want to select multiple countries.

Now, we're ready to remove the missing data in `unemployment`. To do this, we can use the `.dropna()` method.

In [ ]:
unemployment.dropna(subset=['unemployment_rate'], inplace=True)

Note that while we have dropped all observations for which `unemployment_rate == NaN`, this doesn't mean that all of our observations overlap exactly in time. We may find that there are dates where we have data for one country and no data for others.

## Plotting with `pandas`

The best way to get a sense of this data is to plot it. Data visualization is an integral component of conveying your findings to others. While the D-Lab has a [Python Data Visualization](https://github.com/dlab-berkeley/Python-Data-Visualization) workshop, we'll cover some basics in `pandas` here.

Before we begin, let's sort the data by country and date. This is good practice and is especially important when using `pandas`'s `.plot()` method because the x-axis values are based on the indices. When we sort, the index values remain unchanged. Thus, we need to reset them. The `drop` parameter tells `pandas` to construct a `DataFrame` *without* adding a column.

In [ ]:
unemployment.sort_values(['name_en', 'year_month'], inplace=True)
unemployment.reset_index(drop=True, inplace=True)

Let's take a look at Spain's unemployment rate (only because it was the highest) across time.

In [ ]:
spain = unemployment[(unemployment['name_en'] == 'Spain') &
                     (unemployment['seasonality'] == 'sa')]
spain.head()

Now, we'll use the `plot()` function to create a line plot of this data:

In [ ]:
spain['unemployment_rate'].plot(figsize=(10, 8), color='#348ABD')

Note that the values along the x-axis represent the indices associated with Spain in the sorted `unemployment` `DataFrame`. Wouldn't it be nice if, instead, we could show the time period associated with the various unemployment rates for Spain? It might also be interesting to **compare** Spain's unemployment rate with its neighbor to the west, Portugal.

Let's first create a `DataFrame` that contains the unemployment data for both countries.

In [ ]:
ps = unemployment[(unemployment['name_en'].isin(['Portugal', 'Spain'])) &
                  (unemployment['seasonality'] == 'sa')]

Next, we'll generate time series data by converting our years and months into `datetime` objects. `pandas` provides a `to_datetime()` function that makes this relatively simple. It converts an argument&mdash;a single value or an array of values&mdash;to `datetime` (note that the return value [depends on the input](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)). If we were interested in March 23, 1868, for example, we could do the following:

In [ ]:
pd.to_datetime('1868/3/23')

The argument doesn't necessarily have to be specified in the `yyyy/mm/dd` format. You could list it as `mm/dd/yyyy`, but it's a good idea to be explicit. To do so, we pass in a string into the `format` keyword:

In [ ]:
pd.to_datetime('3/23/1868', format='%m/%d/%Y')

Let's create the `datetime` object and add it to the `DataFrame` as a column named `date`. First, we create the datetimes using `pd.to_datetime`, and we'll double check that the values look right:

In [ ]:
datetimes = pd.to_datetime(ps['year'].astype(str) + '/' + ps['month'].astype(str) + '/1')
datetimes.head()

Next, we'll use the `DataFrame.insert()` method to place these datetimes in a new column:

In [ ]:
ps.insert(loc=0, column='date', value=datetimes)

Finally, let's only keep certain columns, rename them, and reshape the `DataFrame`.

In [ ]:
ps = ps[['date', 'name_en', 'unemployment_rate']]
ps.columns = ['Time Period', 'Country', 'Unemployment Rate']

In [ ]:
ps.head()

In order to make the plotting easier, we're going to use a `pivot()` operation to change the structure of the data frame:

In [ ]:
ps = ps.pivot(index='Time Period', columns='Country', values='Unemployment Rate')

In [ ]:
ps.head()

How did the structure of the data frame change?

Now, let's plot:

In [ ]:
ps.plot(figsize=(10, 8), title='Unemployment Rate\n')

---

### Challenge 12: Plot without Missing Values

Note that there are some dates for which we lack data on Spain's unemployment rate. What could you do if you wanted your plot to show only dates where both Spain and Portugal have a defined unemployment rate?

---

In [ ]:
# YOUR CODE HERE